In [41]:
from logicqubit.logic import *
from cmath import *
import numpy as np
import sympy as sp
from scipy.linalg import expm, sinm, cosm, logm, sqrtm

In [42]:
def qft(qr):
    for i in range(len(qr)):
        for j in range(i):
            qr[i].CU1(qr[j], pi/float(2**(i-j)))
        qr[i].H()
        
def iqft(qr):
    for i in range(len(qr)):
        for j in range(i):
            qr[i].CU1(qr[j], -pi/float(2**(i-j)))
        qr[i].H()
        
def swap(s1, s2):
    s2.CX(s1)
    s1.CX(s2)
    s2.CX(s1)
    
def ancilla_rotation(k, C, N, t):
    if k == 0:
        k = 3
    theta = 2*asin(C*N*t / (2*pi*k))
    return theta

In [43]:
# Ax=b, x=3/8=0.375 e -1/8=-0.125

#A = np.matrix([[1.5,0.5],[0.5,1.5]])
A = np.matrix([[3,1],[1,3]])
t0 = 2*pi
#t0 = 0.358166*pi
register_size = 3

# Set C to be the smallest eigenvalue that can be represented by the
C = 2*pi / (2**register_size * t0)

U = np.matrix(expm(1j*A*t0/2))
U_A_div2 = np.matrix(expm(1j*A*t0/2)) 
U_A_div4 = np.matrix(expm(1j*A*t0/4))
U_A_div8 = np.matrix(expm(1j*A*t0/8))
U_A_inv_div2 = np.matrix(expm(-1j*A*t0/2)) 
U_A_inv_div4 = np.matrix(expm(-1j*A*t0/4))
U_A_inv_div8 = np.matrix(expm(-1j*A*t0/8))
print(U_A_div2)

[[1.+4.4408921e-16j 0.+0.0000000e+00j]
 [0.+0.0000000e+00j 1.+4.4408921e-16j]]


In [44]:
b = np.array([1,0])
x = np.linalg.solve(A, b)
print(x)

[ 0.375 -0.125]


In [45]:
logicQuBit  = LogicQuBit(5)

m = 1

a = Qubit()
c1 = Qubit()
c2 = Qubit()
c3 = Qubit()
b = Qubit()

c1.H()
c2.H()
c3.H()

#logicQuBit.PrintState() # a,c1,c2,c3,b
#b.X()
#b.H()

# phase estimation
b.CU(c3, U_A_div8)
b.CU(c2, U_A_div4)
b.CU(c1, U_A_div2)

#b.CU(c3, U_A_div8)
#c3.CU(c2, U_A_div4)
#b.CU(c1, U_A_div2)

b.CU(c1, U)

b.CU(c2, U)
b.CU(c2, U)

b.CU(c3, U)
b.CU(c3, U)
b.CU(c3, U)

iqft([c1,c2,c3])

#logicQuBit.PrintState()

swap(c1,c2)

#a.CRY(c1, ancilla_rotation(1,C,register_size,t0))
#a.CRY(c2, ancilla_rotation(2,C,register_size,t0))
#a.CRY(c3, ancilla_rotation(3,C,register_size,t0))

a.CRY(c1, 4*pi/(2**(m-1)))
a.CRY(c2, 2*pi/(2**(m-1)))
a.CRY(c3, 1*pi/(2**(m-1)))

swap(c1,c2)

qft([c1,c2,c3])

# reverse phase estimation
b.CU(c1, U_A_inv_div2)
b.CU(c2, U_A_inv_div4)
b.CU(c3, U_A_inv_div8)

#b.CU(c1, U_A_div2.conj().T)
#c3.CU(c2, U_A_div4.conj().T)
#b.CU(c3, U_A_div8.conj().T)

c1.H()
c2.H()
c3.H()
#logicQuBit.PrintState()

In [46]:
logicQuBit.getDictPsi()

{'00000': (0.2500000000000002+3.2131722973884535e-16j),
 '00001': (-0.2500000000000003-3.2550159356878817e-16j),
 '00010': (-7.321690718642148e-16+0.24999999999999983j),
 '00011': (-7.093930911542993e-17-0.25000000000000067j),
 '00100': (-1.1865859033808e-16-1.5047887589474974e-16j),
 '00101': (1.3947527204980168e-16+1.377916140651098e-16j),
 '00110': (3.478463511321582e-16-9.11918847638287e-17j),
 '00111': (7.920201690584587e-17+1.358190860390403e-16j),
 '01000': (3.6672643306195197e-16-0.25000000000000017j),
 '01001': (-3.8635258879549916e-16+0.2500000000000002j),
 '01010': (0.2500000000000002+4.641375235333216e-16j),
 '01011': (-0.2500000000000002-3.4736685314507324e-16j),
 '01100': (5.341896931628643e-17+1.440331139973599e-16j),
 '01101': (-9.267128078338084e-17-1.4928834991657525e-16j),
 '01110': (-3.6825450891360043e-16+3.0221235648215795e-17j),
 '01111': (-2.229607773127381e-17-1.5799550988141224e-16j),
 '10000': (0.1249999999999997+0.30177669529663737j),
 '10001': (0.1249999999

In [ ]:
#res_a = logicQuBit.Measure_One(a)
#print(res_a)
res = logicQuBit.Measure([b], True)
logicQuBit.Plot()
print(res)

In [ ]:
def Text2List(table):
    list = [int(i, base=2) for i in table]
    size = len(table[0])
    tmp = sorted(list, key=int, reverse=False)  # values in ascending order
    result = [[int(bin(j)[2:].zfill(size)[i]) for i in range(size)] for j in tmp]
    return result

def BinList(n):
    blist = []
    for i in iter(range(2 ** n)):
        b = bin(i)[2:].zfill(n)  # value in binary, ex: i=1, n=4 -> '0001'
        blist.append(b)
    return Text2List(blist)

#size=3
#blist = [value for i,value in enumerate(BinList(size)) if res[i]>0.1] 
#results = [sum([value/2**(i+1) for i,value in enumerate(j)]) for j in blist]
#print(results)

In [17]:
# todos possíveis valores
size = 3
blist = [[i >> bl & 0x1 for bl in range(size)][::-1] for i in range(2**size)]
results = [2*pi*sum([value/2**(i+1) for i,value in enumerate(j)]) for j in blist]
results

[0.0,
 0.7853981633974483,
 1.5707963267948966,
 2.356194490192345,
 3.141592653589793,
 3.9269908169872414,
 4.71238898038469,
 5.497787143782138]